In [ ]:
%run P1G_1_data_handling.ipynb

## 5. Geo <a class="anchor" id="article_detail"></a>
[Back to Table of Contents](#toc)

(is_pageview AND namespace_id = 0) => page views to articles
(uri_query LIKE '%action=edit%') => desktop wikitext editor
(uri_query LIKE '%action=visualeditor%') => desktop and mobile visualeditor
(uri_query LIKe '%&intestactions=edit&intestactionsdetail=full&uiprop=options%') => mobile wikitext editor

### Editors (and their activity level) across countries <a class="anchor" id="editors_activities_countries"></a>
[Back to Table of Contents](#toc)

In [ ]:
editors_by_activity_level_r = hive.run("""
SELECT country_code,
       wiki_db AS project,
       activity_level,
       SUM(distinct_editors) AS editor_count
FROM   wmf.geoeditors_monthly
WHERE  
    month >= "{Y_START_DATE}"
    AND month <="{TODAY_DATE}"
    AND wiki_db IN ({glow_wiki_dbs})
    AND country_code IN ({glow_iso_codes})
GROUP BY
       country_code, wiki_db, activity_level
""".format(**query_vars))

editors_by_activity_level = editors_by_activity_level_r.copy()

### Editors by localization <a class="anchor" id="edit_localization_editors"></a>
[Back to Table of Contents](#toc)

In [ ]:
eoel_r = hive.run([
    'set hive.groupby.orderby.position.alias = TRUE',
    """
SELECT 
    CASE
        WHEN wiki_db IN ({india_wiki_dbs}) AND country_code IN ({india_country_codes}) THEN 'In_glow_india'
        WHEN wiki_db IN ({india_wiki_dbs}) AND country_code NOT IN ({india_country_codes}) THEN 'Out_glow_india'
        
        WHEN wiki_db IN ({indonesia_wiki_dbs}) AND country_code IN ({indonesia_country_codes}) THEN 'In_glow_indonesia'
        WHEN wiki_db IN ({indonesia_wiki_dbs}) AND country_code NOT IN ({indonesia_country_codes}) THEN 'Out_glow_indonesia'
        
        WHEN wiki_db IN ({argentina_wiki_dbs}) AND country_code IN ({argentina_country_codes}) THEN 'In_glow_argentina'
        WHEN wiki_db IN ({argentina_wiki_dbs}) AND country_code NOT IN ({argentina_country_codes}) THEN 'Out_glow_argentina'
        
        WHEN wiki_db IN ({nigeria_wiki_dbs}) AND country_code IN ({nigeria_country_codes}) THEN 'In_glow_nigeria'
        WHEN wiki_db IN ({nigeria_wiki_dbs}) AND country_code NOT IN ({nigeria_country_codes}) THEN 'Out_glow_nigeria'
        
        WHEN wiki_db IN ({mena_wiki_dbs}) AND country_code IN ({mena_country_codes}) THEN 'In_glow_mena'
        WHEN wiki_db IN ({mena_wiki_dbs}) AND country_code NOT IN ({mena_country_codes}) THEN 'Out_glow_mena'
        ELSE NULL
    END AS editor_location,
    SUM(distinct_editors) AS distinct_editors
FROM wmf.geoeditors_monthly
WHERE 
    month >= "{M_START_DATE}"
    AND month <= "{TODAY_DATE}"
GROUP BY 1

""".format(**query_vars)
])

eoel = eoel_r.copy()

In [ ]:
eoel_piv = hive.run([
    'set hive.groupby.orderby.position.alias = TRUE',
    """
SELECT  wiki_db, distinct_editors,
        CASE WHEN editor_location = 'In' THEN distinct_editors END AS in_region,
        CASE WHEN editor_location = 'Out' THEN distinct_editors END AS out_region
FROM   (SELECT 
        country_code,
        wiki_db,
        CASE
            WHEN wiki_db IN ({india_wiki_dbs}) AND country_code IN ({india_country_codes}) THEN 'In'
            WHEN wiki_db IN ({india_wiki_dbs}) AND country_code NOT IN ({india_country_codes}) THEN 'Out'

            WHEN wiki_db IN ({indonesia_wiki_dbs}) AND country_code IN ({indonesia_country_codes}) THEN 'In'
            WHEN wiki_db IN ({indonesia_wiki_dbs}) AND country_code NOT IN ({indonesia_country_codes}) THEN 'Out'

            WHEN wiki_db IN ({argentina_wiki_dbs}) AND country_code IN ({argentina_country_codes}) THEN 'In'
            WHEN wiki_db IN ({argentina_wiki_dbs}) AND country_code NOT IN ({argentina_country_codes}) THEN 'Out'

            WHEN wiki_db IN ({nigeria_wiki_dbs}) AND country_code IN ({nigeria_country_codes}) THEN 'In'
            WHEN wiki_db IN ({nigeria_wiki_dbs}) AND country_code NOT IN ({nigeria_country_codes}) THEN 'Out'

            WHEN wiki_db IN ({mena_wiki_dbs}) AND country_code IN ({mena_country_codes}) THEN 'In'
            WHEN wiki_db IN ({mena_wiki_dbs}) AND country_code NOT IN ({mena_country_codes}) THEN 'Out'
            ELSE 'no_location_data'
        END AS editor_location,
        SUM(distinct_editors) AS distinct_editors
    FROM wmf.geoeditors_monthly
    WHERE 
        month >= "{M_START_DATE}"
        AND month <= "{TODAY_DATE}"
    GROUP BY 1,2,3) sub
WHERE wiki_db IN ({glow_wiki_dbs}) AND
      country_code IN ({glow_iso_codes})
GROUP BY 1,2,3,4

""".format(**query_vars)
])


In [ ]:
a = eoel_piv.groupby(['wiki_db']).sum()


### Edits by edit localization <a class="anchor" id="edit_localization_editors"></a>
[Back to Table of Contents](#toc)

In [ ]:
# TO DO 
# split out the edit_location column into two columns: IN vs OUT GLOW and location...this will help sort the list

In [ ]:
eel_r2 = hive.run([
    'set hive.groupby.orderby.position.alias = TRUE',
    """
SELECT 
    wiki_db,
    CASE
        WHEN wiki_db IN ({india_wiki_dbs}) AND country_code IN ({india_country_codes}) THEN 'In_glow_india'
        WHEN wiki_db IN ({indonesia_wiki_dbs}) AND country_code IN ({indonesia_country_codes}) THEN 'In_glow_indonesia'
        WHEN wiki_db IN ({argentina_wiki_dbs}) AND country_code IN ({argentina_country_codes}) THEN 'In_glow_argentina' 
        WHEN wiki_db IN ({nigeria_wiki_dbs}) AND country_code IN ({nigeria_country_codes}) THEN 'In_glow_nigeria' 
        WHEN wiki_db IN ({mena_wiki_dbs}) AND country_code IN ({mena_country_codes}) THEN 'In_glow_mena'  
        ELSE NULL
    END AS edit_location,
    (SUM(namespace_zero_edit_count)/SUM(edit_count)) AS  monthly_article_edit_localization_proportion
FROM wmf.geoeditors_edits_monthly
WHERE 
    month >= "{Y_START_DATE}"
    AND month <= "{TODAY_DATE}"
GROUP BY 1, 2
HAVING edit_location IS NOT NULL
""".format(**query_vars)
])

eel2 = eel_r2.copy()


In [ ]:
eel_r2 = '''
SELECT 
    wiki_db,
    CASE
        WHEN wiki_db IN ({india_wiki_dbs}) AND country_code IN ({india_country_codes}) THEN 'In_glow_india'
        WHEN wiki_db IN ({indonesia_wiki_dbs}) AND country_code IN ({indonesia_country_codes}) THEN 'In_glow_indonesia'
        WHEN wiki_db IN ({argentina_wiki_dbs}) AND country_code IN ({argentina_country_codes}) THEN 'In_glow_argentina' 
        WHEN wiki_db IN ({nigeria_wiki_dbs}) AND country_code IN ({nigeria_country_codes}) THEN 'In_glow_nigeria' 
        WHEN wiki_db IN ({mena_wiki_dbs}) AND country_code IN ({mena_country_codes}) THEN 'In_glow_mena'  
        ELSE NULL
    END AS edit_location,
    (SUM(namespace_zero_edit_count)/SUM(edit_count)) AS  monthly_article_edit_localization_proportion
FROM wmf.geoeditors_edits_monthly
WHERE 
    month >= "{Y_START_DATE}"
    AND month <= "{TODAY_DATE}"
GROUP BY wiki_db,
    CASE
        WHEN wiki_db IN ({india_wiki_dbs}) AND country_code IN ({india_country_codes}) THEN 'In_glow_india'
        WHEN wiki_db IN ({indonesia_wiki_dbs}) AND country_code IN ({indonesia_country_codes}) THEN 'In_glow_indonesia'
        WHEN wiki_db IN ({argentina_wiki_dbs}) AND country_code IN ({argentina_country_codes}) THEN 'In_glow_argentina' 
        WHEN wiki_db IN ({nigeria_wiki_dbs}) AND country_code IN ({nigeria_country_codes}) THEN 'In_glow_nigeria' 
        WHEN wiki_db IN ({mena_wiki_dbs}) AND country_code IN ({mena_country_codes}) THEN 'In_glow_mena'  
        ELSE NULL
    END AS edit_location
HAVING edit_location IS NOT NULL
'''

In [ ]:
eel2 = eel2[['wiki_db', 'article_edit_localization_proportion']]


In [ ]:
eel2.to_csv("./data/glow/edit_localization_in_region_wiki.csv", sep=',', encoding = 'utf-8')

### PageViews across countries and wikis <a class="anchor" id="pageviews"></a>
[Back to Table of Contents](#toc)

----------- TO DO --------------

---------- QUESTIONS ------------
1. What is this line for? I don't see the sum that it's getting on the original dataepa :::: 
piv = glow_views.pivot_table("pageviews", index="project", columns="country_code", aggfunc="sum", fill_value=0)

2. Why is the list of wiki projects so small? Something is off in the filtering. 

In [ ]:
#adapted from: https://github.com/wikimedia-research/Editing-movement-metrics/blob/2c261e94ceb4735f53bf51157272e967f5352e94/analysis/diversity-metrics.ipynb

#get data and format df columns
glow_views_r = '''
SELECT project, 
       country_code, 
       SUM(view_count)/12 AS pageviews
FROM wmf.projectview_hourly
WHERE
    CONCAT(year,LPAD(month,2,'0')) >= "{Y_START_DATE_pv}"
    AND CONCAT(year,LPAD(month,2,'0')) < "{TODAY_DATE_pv}"
    access_method in ("mobile web", "mobile app", "desktop") AND
    project in ({glow_projects})
GROUP BY project, country_code
'''

In [ ]:
glow_views.info()

In [ ]:
glow_views_piv = glow_views.pivot_table("pageviews", index="project", columns="country_code", aggfunc="sum", fill_value=0)

In [ ]:
countries_sorted = piv.apply("sum", axis=0).sort_values(ascending=False).index

with pd.option_context(
    'display.max_rows', None, 
    'display.max_columns', None,
    'display.max_colwidth', 500,
):
    display(piv.iloc[:, :25])

In [ ]:
#adapted from: https://github.com/wikimedia-research/Editing-movement-metrics/blob/2c261e94ceb4735f53bf51157272e967f5352e94/analysis/diversity-metrics.ipynb

#get data and format df columns
glow_views_glow_countries_r = '''
SELECT project, 
       country_code, 
       SUM(view_count)/12 AS pageviews
FROM wmf.projectview_hourly
WHERE
    CONCAT(year,LPAD(month,2,'0')) >= "{Y_START_DATE_pv}"
    AND CONCAT(year,LPAD(month,2,'0')) < "{TODAY_DATE_pv}" AND
    access_method in ("mobile web", "mobile app", "desktop") AND
    project in ({glow_projects})
    AND country_code IN ({glow_iso_codes})
GROUP BY project, country_code
'''


In [ ]:
glow_views_glow_countries_r.to_csv("./data/glow/glow_views_glow_countries_yr.csv", sep=',', encoding = 'utf-8')